# Video URL Fetching

In [77]:
import time
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [69]:
# Get chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/jackxu/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
<ipython-input-69-d72d7c643b6e>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [44]:
# Sample URL to channels / playlist
url = "https://www.youtube.com/c/MortdogTFT/videos"

# Load list of Youtuber 
youtuber = pd.read_csv("data/Youtuber_list.csv")

In [66]:
def get_links(url):
    
    driver.get(url)
    # Web scrolling
    ht=driver.execute_script("return document.documentElement.scrollHeight;")
    while True:
        prev_ht=driver.execute_script("return document.documentElement.scrollHeight;")
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2) # Avoid max retreies
        ht=driver.execute_script("return document.documentElement.scrollHeight;")
        if prev_ht==ht: # Reach end of page
            break

    # Result links
    links=driver.find_elements_by_xpath('//*[@id="video-title"]')
    youtube_links = []
    for link in links:
        youtube_links.append(link.get_attribute("href"))
        #print(link.get_attribute("href"))
        
    return youtube_links

In [70]:
youtube_links_set = []
for index, row in youtuber.iterrows():
    youtube_links_set.append([row['Youtuber'], get_links(row['Stream URL'])])

<ipython-input-66-d8f7fa673f90>:15: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  links=driver.find_elements_by_xpath('//*[@id="video-title"]')


In [75]:
youtube_links_set[0][1]

['https://www.youtube.com/watch?v=CWPBiYKb5lo',
 'https://www.youtube.com/watch?v=fIw-lJFDaMk',
 'https://www.youtube.com/watch?v=CZIlvVvCg0Y',
 'https://www.youtube.com/watch?v=a6hUg64Ei7w',
 'https://www.youtube.com/watch?v=wlGOIIH90Uk',
 'https://www.youtube.com/watch?v=X3C5oyKJedY',
 'https://www.youtube.com/watch?v=Y15dMxN2m5E',
 'https://www.youtube.com/watch?v=CeNIglHDeFs',
 'https://www.youtube.com/watch?v=z-Qj71ns3fo',
 'https://www.youtube.com/watch?v=3JlsccealU8',
 'https://www.youtube.com/watch?v=-4CP8gjzwSM',
 'https://www.youtube.com/watch?v=rD3vKKeSei0',
 'https://www.youtube.com/watch?v=b2gzqFPIUJg',
 'https://www.youtube.com/watch?v=BRSpbVg6K3U',
 'https://www.youtube.com/watch?v=0rPdmEHuFMU',
 'https://www.youtube.com/watch?v=DRSCQdaFWZQ',
 'https://www.youtube.com/watch?v=JgA2nRCxL3o',
 'https://www.youtube.com/watch?v=GNWwWTr-8tc',
 'https://www.youtube.com/watch?v=vUtxPWnsvCo',
 'https://www.youtube.com/watch?v=GJGfNwzgWT8',
 'https://www.youtube.com/watch?v=h9VTVm

# Youtube Video Download

In [72]:
from pytube import YouTube

In [73]:
def fetch_video(video_url, download = False, dic = "contents/", 
                res = '480p', file_ex = 'mp4'):
    
    yt = YouTube(video_url)

    # Select format
    yss = yt.streams.filter(only_video=True, file_extension=file_ex, res=res)
    
    if len(yss) != 0:
        ys = yt.streams.get_by_itag(yss[-1].itag)
        fps, itag = ys.fps, ys.itag
        
        # Download videos
        if download:
            ys.download(dic)
    else:
        fps, itag, ys = None, None, None

    return [yt.title, video_url, yt.publish_date, yt.views,  
            yt.rating, yt.keywords, ys, fps, itag]

In [95]:
youtube_object_set = []

In [110]:
youtube_data = []
date_cap = datetime.date(2021, 10, 3)
i = 0

# Fetch video
for l in youtube_links_set[i][1]:
    youtube_info = fetch_video(l)
    youtube_data.append(youtube_info)
    
    # Stop when date cap is reached
    if youtube_info[2].date() < date_cap:
        break
    
video_info = pd.DataFrame(youtube_data, columns=['Title', 'URL', 'Publish Date',
                                      'Views', 'Rating', 'Keywords', 'Streams', 'FPS', 'Itag'])

In [111]:
# Add object to the list
youtube_object_set.append(youtube_data)

In [112]:
video_info.insert(loc=0, column='Author', value=youtube_links_set[i][0])
video_info.to_csv("data/" + youtube_links_set[i][0] + "_Youtube_data.csv", index=False)

#### Saving objects to file with Pickle

In [114]:
import pickle

with open('data/youtube_object_set.pkl', 'wb') as youtube_object_set_file:
  pickle.dump(youtube_object_set, youtube_object_set_file)

with open('data/youtube_links_set.pkl', 'wb') as youtube_links_set_file:
  pickle.dump(youtube_links_set, youtube_links_set_file)

In [120]:
len(youtube_object_set[3])

9

### Extra Curiocities

In [93]:
df = pd.DataFrame('x', index=range(4), columns=list('ABC'))
df.loc[-1,:] = 'y'
df.index = df.index + 1
df

,A,B,C
1,x,x,x
2,x,x,x
3,x,x,x
4,x,x,x
0,y,y,y
